# Baseline CNN Model for American Sign Language Classification

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
from tensorflow.keras import activations

### Part One: Data Loader

In [2]:
dict_labels = {
    '0':1, '1':2, '2':3, '3':4, '4':5, '5':6, '6':7, '7':8, '8':9, '9':10,
    'a':11, 'b':12, 'c':13, 'd':14, 'e':15, 'f':16, 'g':17, 'h':18, 'i':19,
    'j':20, 'k':21, 'l':22, 'm':23, 'n':24, 'o':25, 'p':26, 'q':27, 'r':28,
    's':29, 't':30, 'u':31, 'v':32, 'w':33, 'x':34, 'y':35, 'z':36, 
}
img_size = 50

In [3]:
x = []
y = []
path = 'Data/asl/train_asl/'
gestures = os.listdir(path)[1:]

for ix in gestures:
    images = os.listdir(path +ix)
    for cx in images:
        img_path = path+'/' +ix + '/' + cx
        img = cv2.imread(img_path, 1)
        img = cv2.resize(img, (img_size,img_size), interpolation = cv2.INTER_AREA) 
        
        img = img.reshape((img_size, img_size,3))
        img = np.expand_dims(img, axis=0)
        
        datagen = ImageDataGenerator(horizontal_flip=True)
        transformed_img = datagen.flow(img, batch_size=1)
        for i in range(2):
            image = next(transformed_img)[0].astype('uint8')
            x.append(image)
            y.append(dict_labels[ix])

In [4]:
Y = to_categorical(np.array(y))
X = np.array(x)/255.0 # normalization

In [5]:
# test set
x_test = []
y_test = []
path_test = 'Data/asl/test_asl/'
gestures_test = os.listdir(path_test)[1:]

for ix in gestures_test:
    images = os.listdir(path_test +ix)
    for cx in images:
        img_path = path_test+'/' +ix + '/' + cx
        img = cv2.imread(img_path, 1)
        img = cv2.resize(img, (img_size,img_size), interpolation = cv2.INTER_AREA) 

        x_test.append(img)
        y_test.append(dict_labels[ix])
        
Y_test = to_categorical(np.array(y_test))
X_test = np.array(x_test)/255.0   # normalization     


In [6]:
# split train data into train set and validation set
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2,shuffle=True)

### small dataset

In [7]:
dict_labels_new = {
    '0':1, '1':2, '2':3, '3':4, '4':5, '5':6, '6':7, '7':8, '8':9, '9':10,
    'A':11, 'B':12, 'C':13, 'D':14, 'E':15, 'F':16, 'G':17, 'H':18, 'I':19,
    'J':20, 'K':21, 'L':22, 'M':23, 'N':24, 'O':25, 'P':26, 'Q':27, 'R':28,
    'S':29, 'T':30, 'U':31, 'V':32, 'W':33, 'X':34, 'Y':35, 'Z':36, 
}

In [8]:
xs = []
ys = []
path = 'Gesture/GestureImageData/'
gestures = os.listdir(path)[1:]


for ix in gestures:
    if(ix!='_'):
        images = os.listdir(path +ix)
        for cx in images:
            img_path = path+'/' +ix + '/' + cx
            img = cv2.imread(img_path, 1)
            img = cv2.resize(img, (img_size,img_size), interpolation = cv2.INTER_AREA) 

            img = img.reshape((img_size, img_size,3))
            img = np.expand_dims(img, axis=0)
        
            datagen = ImageDataGenerator(horizontal_flip=True)
            transformed_img = datagen.flow(img, batch_size=1)
            for i in range(2):
                image = next(transformed_img)[0].astype('uint8')
                xs.append(image)
                ys.append(dict_labels_new[ix])

In [9]:
Ys_temp = to_categorical(np.array(ys))
Xs_temp = np.array(xs)/255.0

In [10]:
Xs, Xs_test, Ys, Ys_test = train_test_split(Xs_temp, Ys_temp, test_size=0.2,shuffle=True)

In [12]:
Xs_train, Xs_val, Ys_train, Ys_val = train_test_split(Xs, Ys, test_size=0.2,shuffle=True)

### merged dataset

In [13]:
x_train_merged = tf.concat([X_train, Xs_train], axis=0)
x_test_merged = tf.concat([X_test, Xs_test], axis=0)
x_val_merged = tf.concat([X_val, Xs_val], axis=0)

y_train_merged = tf.concat([Y_train, Ys_train], axis=0)
y_test_merged = tf.concat([Y_test, Ys_test], axis=0)
y_val_merged = tf.concat([Y_val, Ys_val], axis=0)

In [14]:
x_train_merged.shape

TensorShape([101465, 50, 50, 3])

### Part Two: Build the CNN Model 
Referencing the CNN model described in the [Using Deep Convolutional Networks for Gesture Recognition in American Sign Language](https://arxiv.org/pdf/1710.06836.pdf) paper.

In [15]:
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), input_shape=(img_size, img_size, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(32, (3, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3)),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128),
        layers.Activation(activations.relu),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(37, activation='softmax')
    ]
)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


In [17]:
history = model.fit(x_train_merged, y_train_merged, batch_size=128, epochs=50, validation_data = (x_val_merged,y_val_merged),shuffle=True)


Epoch 1/50
793/793 [==============================] - 803s 1s/step - loss: 1.3244 - accuracy: 0.6057 - val_loss: 0.3493 - val_accuracy: 0.8722
Epoch 2/50
793/793 [==============================] - 778s 980ms/step - loss: 0.3910 - accuracy: 0.8667 - val_loss: 0.1367 - val_accuracy: 0.9516
Epoch 3/50
793/793 [==============================] - 714s 900ms/step - loss: 0.2452 - accuracy: 0.9127 - val_loss: 0.1390 - val_accuracy: 0.9457
Epoch 4/50
793/793 [==============================] - 785s 990ms/step - loss: 0.1809 - accuracy: 0.9346 - val_loss: 0.0833 - val_accuracy: 0.9680
Epoch 5/50
793/793 [==============================] - 809s 1s/step - loss: 0.1484 - accuracy: 0.9466 - val_loss: 0.0821 - val_accuracy: 0.9696
Epoch 6/50
793/793 [==============================] - 729s 920ms/step - loss: 0.1258 - accuracy: 0.9547 - val_loss: 0.5398 - val_accuracy: 0.8476
Epoch 7/50
793/793 [==============================] - 739s 932ms/step - loss: 0.1101 - accuracy: 0.9601 - val_loss: 0.0633 - val_a

In [18]:
model.save("merged dataset with aug")


INFO:tensorflow:Assets written to: merged dataset with aug/assets


In [19]:
# save the history for plotting:
f = open('history_merged_data_withaug.pckl', 'wb')
pickle.dump(history.history, f)
f.close()


### Load Model

In [24]:
no_aug_model = keras.models.load_model("dataset without aug")

In [25]:
no_aug_scores = no_aug_model.evaluate(X_test, Y_test)

159/159 [==============================] - 5s 29ms/step - loss: 0.0233 - accuracy: 0.9921


In [26]:
aug_model = keras.models.load_model("dropout batchnorm with aug")

In [27]:
aug_scores = aug_model.evaluate(X_test, Y_test)


159/159 [==============================] - 7s 41ms/step - loss: 0.0142 - accuracy: 0.9965


In [28]:
aug_nbn_model = keras.models.load_model("dropout with aug")
aug_nbn_scores = aug_nbn_model.evaluate(X_test, Y_test)

159/159 [==============================] - 4s 26ms/step - loss: 0.0448 - accuracy: 0.9813
